## 準備

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, countDistinct, when, broadcast, count, row_number, substring,udf,dense_rank,format_string,first,lit, to_date, transform,collect_set
from pyspark.sql.window import Window
import unicodedata
from pyspark.sql.types import StringType
import numpy as np
import pyspark.sql.functions as F


# Sparkセッションがまだ存在しない場合に作成
import sys
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("LoadParquetFile")
    .config("spark.executor.memory", "32g")
    .config("spark.driver.memory", "16g")
    .config("spark.sql.shuffle.partitions", "500")
    .config("spark.sql.files.maxPartitionBytes", "32m")
    # ここを追加（Driver/Executor ともに同じ Python を使わせる）
    .config("spark.pyspark.driver.python", sys.executable)
    .config("spark.pyspark.python", sys.executable)
    .getOrCreate()
)

file_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/data/chart_disease_merge/final_chart_patient_disease_with_icd.parquet"
soap_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/data/SOAP/preprocessed_soap"
lab_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/data/data2/data_for_model_with_dz_ord_by_chart_and_ordDisease.parquet"
ch_pat_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/data/row/chart_patient.csv"
order_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/data/order_data_processed/grouped_recipe_shinsatsu_filtered.parquet"

df = spark.read.parquet(file_path)
soap_df = spark.read.parquet(soap_path)
df_lab = spark.read.parquet(lab_path)
ch_pat_df = spark.read.option("header",True).csv(ch_pat_path)
order_df = spark.read.parquet(order_path)
order_df = order_df.filter(F.col("shinsatsu") == 1) 
order_df = order_df.drop("create_datetime")

@F.udf("string")
def norm(s):
    if s is None:
        return None
    s = unicodedata.normalize("NFKC", s).strip().lower()
    # 追加の正規化ルールがあればここで
    return s or None

25/11/12 09:30:07 WARN Utils: Your hostname, Donuts-NM3044-2508.local resolves to a loopback address: 127.0.0.1; using 10.11.34.151 instead (on interface en0)
25/11/12 09:30:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/12 09:30:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 56488)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.fra

In [2]:
import json

with open("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/医療シソーラス/synonym_to_icd.json", "r", encoding="utf-8") as f:
    synonym_to_icd = json.load(f)

from pyspark.sql import functions as F
df = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")
from pyspark.sql import functions as F, Window

df_tok = (
    spark.read
         .parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")
         .select("chart_id", "soap_tokens")
)


In [3]:
from pyspark.sql import functions as F

# 正規化 → explode
tok_norm = (
    df_tok
    .select(
        "chart_id",
        F.when(F.col("soap_tokens").isNull(), F.array().cast("array<string>"))
         .otherwise(F.col("soap_tokens")).alias("soap_tokens")
    )
    .select(
        "chart_id",
        F.expr("transform(soap_tokens, x -> trim(lower(x)))").alias("soap_tokens")
    )
)

tok_expl = (
    tok_norm
    .select("chart_id", F.explode("soap_tokens").alias("token"))
    .filter(F.col("token").isNotNull())
    .select("chart_id", F.col("token").cast("string").alias("token"))
)

# 一度だけ保存（パスは任意）
out_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/tok_expl.parquet"
# tok_expl.write.mode("overwrite").parquet(out_path)

# 以後は読み直して使う
tok_expl_saved = spark.read.parquet(out_path)
# tok_expl_saved.show(10, truncate=False)

# 2) token ごとに代表 chart_id を1つだけ残す（ここでは最小値）
tok_dedup = (
    tok_expl_saved
      .groupBy("token")
      .agg(F.min("chart_id").alias("chart_id"))
)

# tok_dedup.show(20, truncate=False)

# # 保存するなら
out_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/tok_unique_token.parquet"
# tok_dedup.write.mode("overwrite").parquet(out_path)

tok_dedup = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/tok_unique_token.parquet")
tok_dedup.show(10, truncate=False)


+--------------+--------+
|token         |chart_id|
+--------------+--------+
|r1            |22318792|
|倦怠感        |22417498|
|2回目         |22418736|
|なっ          |21663439|
|ホルネル症候群|22428691|
|日々          |22423552|
|serm          |22434945|
|抗凝固薬      |21963424|
|皮膚乾燥      |22426470|
|フラフラ      |22428657|
+--------------+--------+
only showing top 10 rows



In [4]:
import ast
import numpy as np
import pandas as pd

def normalize_aliases(value):
    if value is None:
        return []
    if isinstance(value, (list, tuple)):
        return list(value)
    if isinstance(value, np.ndarray):
        return value.tolist()
    if isinstance(value, str):
        stripped = value.strip()
        if not stripped:
            return []
        if stripped.startswith("[") and stripped.endswith("]"):
            try:
                return ast.literal_eval(stripped)
            except (ValueError, SyntaxError):
                pass
        return [stripped]
    return []

node_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/nodes_v4.parquet"
node_df = pd.read_parquet(node_path)

alias_series = (
    node_df["aliases"]
    .map(normalize_aliases)
    .explode()
    .dropna()
    .astype(str)
    .str.strip()
)
alias_series = alias_series[alias_series != ""]

unique_aliases = sorted(alias_series.unique())
alias_df = pd.DataFrame(unique_aliases, columns=["alias"])
# alias_df.to_csv("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/aliases.csv", index=False)

## matchしたentityのリストを作成

In [31]:
import pandas as pd
import unicodedata as ud

# ▼ 読み込み（そのまま）
token = pd.read_parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/tok_unique_token.parquet")
node = pd.read_csv("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/aliases.csv")
node = node.drop_duplicates(subset=['alias'])

# ▼ 正規化関数：NFKCで全角半角を統一、前後空白除去、casefoldで大文字小文字同一視
#   ※ 完全一致にしたい場合は、下の normalize を単に str(s).strip() にするなど調整してください
def normalize(s):
    if pd.isna(s):
        return None
    return ud.normalize("NFKC", str(s)).strip().casefold()

# 比較用キーを付与
node["_norm"]  = node["alias"].map(normalize)
token["_norm"] = token["token"].map(normalize)

# 同一ノーマライズ値の重複は先勝ちで整理（必要に応じて外してOK）
node_key  = node.drop_duplicates(subset=["_norm"], keep="first")
token_key = token.drop_duplicates(subset=["_norm"], keep="first")

# 外部結合で突き合わせ
merged = (
    node_key[["alias", "_norm"]]
    .merge(
        token_key[["token", "_norm"]],
        on="_norm",
        how="outer",
        suffixes=("_node", "_token"),
        indicator=True
    )
)

# ステータス付与
status_map = {
    "both": "match",
    "left_only": "only node(alias)",
    "right_only": "only token(token)",
}
merged["status"] = merged["_merge"].map(status_map)

# 見やすい列順に整形
result = merged.rename(columns={"alias": "alias(node)", "token": "token(token)"}).loc[
    :, ["alias(node)", "token(token)", "status", "_norm"]
].sort_values(["status", "_norm"], kind="stable").reset_index(drop=True)

# 結果の確認（件数）
summary = result["status"].value_counts(dropna=False)

result[result['status'] == 'match'].to_parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/mached_alias_inSOAP.parquet")

## matched_token_listを使用して、aliasをSOAPから抽出

In [33]:
from pyspark.sql import functions as F, types as T
import unicodedata as ud

# --- 正規化UDF（NFKC + strip + casefold） ---
@F.udf(returnType=T.StringType())
def normalize_udf(s):
    if s is None:
        return None
    return ud.normalize("NFKC", str(s)).strip().casefold()


matched_token = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/mached_alias_inSOAP.parquet")

norm = normalize_udf

# 1) alias 側：正規化キーを作ってユニーク化
alias_norm = (
    matched_token
    .select(F.col("`alias(node)`").alias("alias"))
    .dropna()
    .dropDuplicates()
    .withColumn("norm", normalize_udf(F.col("alias")))
    .dropna(subset=["norm"])
    .dropDuplicates(["norm"])
    .select("norm")
)

# 2) df_tok 側：各行の soap_tokens（array<string>）を展開→正規化
df_tok_with_id = df_tok.withColumn("__rowid", F.monotonically_increasing_id())

tok_exploded = (
    df_tok_with_id
    .withColumn("token", F.explode_outer(F.col("soap_tokens")))   # null/空配列もケア
    .withColumn("norm", normalize_udf(F.col("token")))
)

# 3) 正規化キーで内部結合（ヒットのみ）→ 行IDごとにマッチしたトークンをまとめる
hits = (
    tok_exploded.join(alias_norm, on="norm", how="inner")
    .groupBy("__rowid")
    .agg(F.collect_set("token").alias("matched_aliases"))  # 重複はまとめるなら collect_set
)

# 4) 元DFに戻す（null は空配列に）
result = (
    df_tok_with_id
    .join(hits, on="__rowid", how="left")
    .withColumn(
        "matched_aliases",
        F.when(F.col("matched_aliases").isNull(),
               F.lit([]).cast(T.ArrayType(T.StringType()))  # 空配列
        ).otherwise(F.col("matched_aliases"))
    )
    .withColumn("matched_count", F.size(F.col("matched_aliases")))
    .withColumn("has_match", F.col("matched_count") > 0)
    .drop("__rowid")
)

# 例：先頭数行を確認（Notebook等で）
# result.select("soap_tokens", "mtched_aliases", "matched_count", "has_match").show(truncate=False)
result.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/soap_tokens_with_matched_aliases.parquet")


## match したトークン

In [61]:

matched_token = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/soap_tokens_with_matched_aliases.parquet")
aliases_long = matched_token.select(
    F.col("chart_id"),
    F.explode_outer("matched_aliases").alias("alias")
).select(
    "chart_id",
    norm(F.col("alias")).alias("alias")
).where(F.col("alias").isNotNull()).dropDuplicates()

# aliases_long.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_long.parquet")
# aliases_long = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_long.parquet")


In [11]:
from pyspark.sql import functions as F

triples = (
    spark.read
         .csv("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/merged_triples_all_v3.csv", header=True, inferSchema=True)
         .filter(F.col("relation") == "symptom")
)

# 1) 正規化
t = (
    triples
    .withColumn("head_norm", norm(F.col("head")))
    .withColumn("tail_norm", norm(F.col("tail")))
)

# 2) 両向きに展開して「病名->症状」だけを持つテーブルに揃える
#    (head, tail) → (disease=head, symptom=tail) と (disease=tail, symptom=head)
sym2dis_a = (
    t.select(
        F.col("head_norm").alias("disease_norm"),
        F.col("tail_norm").alias("symptom_norm")
    )
)

sym2dis_b = (
    t.select(
        F.col("tail_norm").alias("disease_norm"),
        F.col("head_norm").alias("symptom_norm")
    )
)

# 3) 結合・クリーニング
sym2dis = (
    sym2dis_a.unionByName(sym2dis_b)
    .filter(
        (F.col("disease_norm").isNotNull()) & (F.col("disease_norm") != "") &
        (F.col("symptom_norm").isNotNull()) & (F.col("symptom_norm") != "")
    )
    # 同一語同士の自己ループを落としたい場合（任意）
    .filter(F.col("disease_norm") != F.col("symptom_norm"))
    .distinct()
    .cache()
)
_ = sym2dis.count()  # cache warm-up


In [12]:
from pyspark.sql import functions as F

# 病名ごとの紐づく症状数とエッジ数（重複あり/なし）
dis_summary = (
    sym2dis
    .groupBy("disease_norm")
    .agg(
        F.count("*").alias("edges"),                              # 行数（重複あり）
        F.countDistinct("symptom_norm").alias("n_symptoms")       # ユニーク症状数
    )
    .orderBy(F.desc("n_symptoms"), F.asc("disease_norm"))
)

dis_summary.show(50, truncate=False)  # 上位だけ見る


+-------------------------------+-----+----------+
|disease_norm                   |edges|n_symptoms|
+-------------------------------+-----+----------+
|徴候と症状                     |858  |858       |
|発熱                           |402  |402       |
|疾患                           |391  |391       |
|疼痛                           |323  |323       |
|よう                           |307  |307       |
|無症候性疾患                   |251  |251       |
|頭痛                           |233  |233       |
|嘔吐                           |211  |211       |
|腹痛                           |204  |204       |
|悪心                           |193  |193       |
|ヒト免疫不全ウイルス(hiv)感染症|192  |192       |
|圧痛                           |174  |174       |
|疲労                           |174  |174       |
|呼吸困難                       |170  |170       |
|筋力低下                       |169  |169       |
|心不全                         |153  |153       |
|下痢                           |150  |150       |
|腫脹                           |143  |

In [13]:
from pyspark.sql import functions as F

# diseaseごとのユニーク症状数
dis_symcnt = (
    sym2dis
    .groupBy("disease_norm")
    .agg(F.countDistinct("symptom_norm").alias("n_symptoms"))
)

# 50以上を除外
to_drop = dis_symcnt.filter(F.col("n_symptoms") >= 50).select("disease_norm")

sym2dis_filtered = (
    sym2dis.join(to_drop, "disease_norm", "left_anti")  # ← ここで除外
).cache()

# 確認（任意）
# sym2dis_filtered.select("disease_norm").distinct().count()
# sym2dis_filtered.show(10, truncate=False)


In [15]:

# aliases 側の正規化
aliases_clean = (
    aliases_long
    .withColumn("alias_norm", norm(F.col("alias")))
    .filter(F.col("alias_norm").isNotNull() & (F.col("alias_norm") != ""))
)

In [16]:
aliases_clean.show(10)

+--------+--------+----------+
|chart_id|   alias|alias_norm|
+--------+--------+----------+
|23726950|副鼻腔炎|  副鼻腔炎|
|31276479|中途覚醒|  中途覚醒|
|35004503|  むくみ|    むくみ|
|29802764|インフル|  インフル|
|25135741|ヘルニア|  ヘルニア|
|31538416|喉の痛み|  喉の痛み|
|22996620|    白苔|      白苔|
|23406797|    心配|      心配|
|29929631|精神症状|  精神症状|
|34696063|    痒み|      痒み|
+--------+--------+----------+
only showing top 10 rows



## SOAPの症状とKGの症状の一致度から病名を推測

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window as W

# alias(=症状) → 病名 を引く
hit = (
    aliases_clean.join(sym2dis_filtered, aliases_clean.alias_norm == sym2dis_filtered.symptom_norm, "inner")
                 .select("chart_id", "disease_norm")
)

# chart ごとの病名ランキング（件数）
rank_count = (
    hit.groupBy("chart_id", "disease_norm")
       .agg(F.count(F.lit(1)).alias("score_cnt"))
)

# 見やすく上位だけ
w = W.partitionBy("chart_id").orderBy(F.desc("score_cnt"), F.asc("disease_norm"))
rank_top = (
    rank_count.withColumn("rk", F.row_number().over(w))
              .filter(F.col("rk") <= 10)
              .orderBy("chart_id", "rk")
)

rank_top.show(truncate=False)


In [ ]:
from pyspark.sql import functions as F

grouped = (
    rank_top
    .groupBy("chart_id")
    .agg(
        F.sort_array(
            F.collect_list(
                F.struct("rk", F.col("disease_norm").alias("disease"), F.col("score_cnt").alias("score"))
            ),
            asc=True  # rk 昇順
        ).alias("candidates")
    )
)

grouped.show(truncate=False)


In [ ]:
grouped.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/symptom_to_disease_candidates.parquet")

## 推測した病名と実際の登録されている病名の比較

In [5]:
label = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_clean_matched_icd.parquet")
from pyspark.sql import functions as F

filtered = (
    label
    .join(grouped.select("chart_id").dropDuplicates(), on="chart_id", how="inner")
    .select("chart_id", "main_name_list", "suspect_name_list", "acute_name_list", "treat_name_list")
)


NameError: name 'grouped' is not defined

In [ ]:
t = label.select("chart_id", "main_name_list", "suspect_name_list", "acute_name_list", "treat_name_list","soap_text")
# t.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/label_with_soap.parquet")

In [ ]:
from pyspark.sql import functions as F

# grouped: chart_id, candidates(array<struct<rk:int, disease:string, score:double>>)
# filtered: 少なくとも chart_id 列を持つ

result = (
    filtered.alias("f")
    .join(grouped.select("chart_id", "candidates").alias("g"), on="chart_id", how="left")
    # .na.fill({"candidates": ???})  # 配列のデフォルト値は面倒なので null のままでもOK
)
# 見る
result.show(truncate=False)


## matched_tokenごとのカウント

In [64]:
aliases_long = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_long.parquet")
# aliases_long = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_long.parquet")
from pyspark.sql import functions as F

# Drop noisy tokens from aliases_long
bad_aliases = ["g", "c", "p", "よう", "らい","体重","症状","疾患","重症化","アレルギー","妊娠","増悪","病気"]

aliases_long = aliases_long.filter(~F.col("alias").isin(bad_aliases))

In [66]:
from pyspark.sql import functions as F
# aliases_long = matched_token.select(
#     F.col("chart_id"),
#     F.explode_outer("matched_aliases").alias("alias")
# ).select(
#     "chart_id",
#     norm(F.col("alias")).alias("alias")
# ).where(F.col("alias").isNotNull()).dropDuplicates()

# # aliases 側の正規化
# aliases_clean = (
#     aliases_long
#     .withColumn("alias_norm", norm(F.col("alias")))
#     .filter(F.col("alias_norm").isNotNull() & (F.col("alias_norm") != ""))
# )
# 基本（頻度順）
alias_cnt = (
    aliases_long
    .groupBy("alias")
    .count()
    .withColumn("alias_len", F.length(F.col("alias")))
    .orderBy(F.desc("count"))
)

alias_cnt.show(truncate=False)
alias_cnt.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/alias_frequency.parquet")


+----------------+------+---------+
|alias           |count |alias_len|
+----------------+------+---------+
|発熱            |610944|2        |
|痛み            |392847|2        |
|鼻水            |338321|2        |
|咽頭痛          |298152|3        |
|covid19         |265440|7        |
|コロナ          |242713|3        |
|発赤            |212522|2        |
|インフルエンザ  |205529|7        |
|花粉症          |203844|3        |
|鼻汁            |202344|2        |
|痰              |197974|1        |
|咽頭発赤        |193698|4        |
|xp              |185662|2        |
|圧痛            |185103|2        |
|熱              |175123|1        |
|喉の痛み        |169738|4        |
|かゆみ          |165073|3        |
|下痢            |157690|2        |
|白苔            |146293|2        |
|アレルギー性鼻炎|141158|8        |
+----------------+------+---------+
only showing top 20 rows



In [67]:
import ast
import numpy as np
import pandas as pd

def normalize_aliases(value):
    if value is None:
        return []
    if isinstance(value, (list, tuple)):
        return list(value)
    if isinstance(value, np.ndarray):
        return value.tolist()
    if isinstance(value, str):
        stripped = value.strip()
        if not stripped:
            return []
        if stripped.startswith("[") and stripped.endswith("]"):
            try:
                parsed = ast.literal_eval(stripped)
                return parsed if isinstance(parsed, (list, tuple)) else [stripped]
            except (ValueError, SyntaxError):
                return [stripped]
        return [stripped]
    return []

alias_map_df = (
    node_df[["Name", "aliases"]]
    .assign(
        Name=lambda df: df["Name"].astype(str).str.strip(),
        aliases=lambda df: df["aliases"].map(normalize_aliases),
    )
    .explode("aliases")
    .dropna(subset=["aliases"])
    .assign(alias=lambda df: df["aliases"].astype(str).str.strip())
    .query("alias != ''")
    .query("alias != Name")              # Name と同じ alias を除外
    .drop_duplicates(subset=["alias"], keep="first")
    [["alias", "Name"]]
)

alias_to_name = dict(zip(alias_map_df["alias"], alias_map_df["Name"]))
print(f"alias entries: {len(alias_to_name)}")
alias_map_df.head()


alias entries: 98855


,alias,Name
6,11ベータ水酸化酵素欠損症,11β−水酸化酵素欠損症
6,11β-ハイドロキシラーゼ欠損症,11β−水酸化酵素欠損症
9,17アルファ水酸化酵素欠損症,17α−水酸化酵素欠損症
9,17α-ハイドロキシラーゼ欠損症,17α−水酸化酵素欠損症
11,18ハイドロキシラーゼ欠損症,18−水酸化酵素欠損症


In [68]:
from pyspark.sql import functions as F

# 1) pandas -> Spark DataFrame（alias_norm, canonical_name）
alias_map_sdf = spark.createDataFrame(
    [(alias, name) for alias, name in alias_to_name.items()],
    schema=["alias", "alias_mapped"]
)

# 2) alias_cnt にマッピング列を追加（ヒットしなければ alias_norm のまま）
alias_cnt_mapped = (
    alias_cnt.alias("cnt")
    .join(alias_map_sdf.alias("map"), on="alias", how="left")
    .withColumn("alias_mapped", F.coalesce(F.col("alias_mapped"), F.col("alias")))
)

from pyspark.sql import functions as F

aliases_long_mapped = (
    aliases_long.alias("al")
    .join(alias_map_sdf.alias("map"), on="alias", how="left")
    .withColumn("alias_mapped", F.coalesce(F.col("map.alias_mapped"), F.col("al.alias")))
)


from pyspark.sql import functions as F

aliases_by_chart = (
    aliases_long_mapped
    .groupBy("chart_id")
    .agg(
        F.collect_set("alias").alias("aliases_raw"),
        F.collect_set("alias_mapped").alias("aliases_mapped"),
        F.countDistinct("alias_mapped").alias("n_aliases")
    )
)


In [72]:
from pyspark.sql import functions as F

alias_mapped_cnt = (
    aliases_long_mapped
    .filter(F.col("alias_mapped").isNotNull() & (F.col("alias_mapped") != ""))
    .groupBy("alias_mapped")
    .count()
    .orderBy(F.desc("count"), F.asc("alias_mapped"))
)

alias_mapped_cnt.show(50, truncate=False)


+-------------------+------+
|alias_mapped       |count |
+-------------------+------+
|発熱               |846974|
|鼻汁               |560049|
|疼痛               |553859|
|呼吸音             |394595|
|そう痒症           |384966|
|インフルエンザ-ヒト|381864|
|咽頭痛             |298408|
|発赤               |273310|
|covid19            |265440|
|喀痰               |250557|
|COVID-19           |249161|
|圧痛               |231232|
|下痢               |206545|
|かぜ               |205948|
|花粉症             |204562|
|咽頭発赤           |193698|
|腫脹               |192724|
|xp                 |185662|
|悪心               |181034|
|腹痛               |170189|
|咽喉痛             |169911|
|白色苔癬           |146298|
|鼻炎-アレルギー性  |142973|
|鼻閉               |140429|
|扁桃肥大           |140334|
|尋常性ざ瘡         |140063|
|急性上気道炎       |134897|
|疲労               |133336|
|mg                 |132276|
|発疹               |131225|
|うつ状態           |127553|
|呼吸困難           |122536|
|倦怠感             |122181|
|湿疹               |115101|
|過敏症-薬物        |108292|


In [ ]:
# aliases_by_chart.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_by_chart.parquet")

## labelとmatch_tokenの関係

In [5]:
label = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/label_with_soap.parquet")
matched_entity_by_soap  = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/aliases_by_chart.parquet")

merged_df = (
    matched_entity_by_soap.alias("m")
    .join(label.alias("l"), on="chart_id", how="inner")
)

merged_df.show(5, truncate=False)
# merged_df.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/label_with_matched_entity.parquet")

+--------+--------------------------------------------------+-----------------------------------------------------------------------------+---------+------------------------------+----------------------------+---------------+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
from pyspark.sql import functions as F
from pyspark.sql import Window

def rank_aliases_by_disease(
    df,
    disease_col: str = "main_name_list",   # 例: "main_name_list", "suspect_name_list" など
    alias_col: str = "aliases_raw",        # 例: "aliases_raw", "aliases_mapped"
    normalize_alias: bool = True,          # 大文字小文字/前後空白の正規化をするか
    drop_null_alias: bool = True,          # None/空文字aliasを除外するか
    top_k: int | None = None               # 病名ごとの上位K件だけ返したい場合は整数指定
):
    """
    df: Spark DataFrame
    戻り値: columns = [disease, alias, cnt, rank]
    """

    # --- 入力の安全化: null配列→空配列
    safe = df.withColumn(disease_col, F.coalesce(F.col(disease_col), F.array())) \
             .withColumn(alias_col,   F.coalesce(F.col(alias_col),   F.array()))

    # --- explode（病名×alias の直積ではなく、それぞれの行内で別々にexplode）
    #     「同一レコード内での病名とaliasの共起」を数える設計。
    exploded = (safe
        .withColumn("disease", F.explode(F.col(disease_col)))
        .withColumn("alias",   F.explode(F.col(alias_col)))
    )

    # --- 文字列正規化（任意）
    if normalize_alias:
        exploded = exploded.withColumn("alias",
            F.when(F.col("alias").isNotNull(), F.lower(F.trim(F.col("alias")))).otherwise(F.col("alias"))
        )
    exploded = exploded.withColumn("disease", F.trim(F.col("disease")))

    # --- 無効値の除去（任意）
    if drop_null_alias:
        exploded = exploded.filter(
            (F.col("alias").isNotNull()) & (F.col("alias") != "") &
            (F.col("disease").isNotNull()) & (F.col("disease") != "")
        )

    # --- 集計：病名×aliasごとの出現回数
    agg = (exploded
        .groupBy("disease", "alias")
        .agg(F.count(F.lit(1)).alias("cnt"))
    )

    # --- 病名ごとにカウント降順で順位付け
    w = Window.partitionBy("disease").orderBy(F.col("cnt").desc(), F.col("alias").asc())
    ranked = agg.withColumn("rank", F.dense_rank().over(w))

    # --- 上位K件だけに絞る（任意）
    if top_k is not None:
        ranked = ranked.filter(F.col("rank") <= F.lit(top_k))

    # 表示順の最終整形
    result = ranked.select("disease", "alias", "cnt", "rank").orderBy("disease", "rank", F.col("alias").asc())
    return result


In [7]:
# 1) main_name_list × aliases_raw のランキング（各病名ごとに上位10件）
merged_df = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/label_with_matched_entity.parquet")
top_raw = rank_aliases_by_disease(merged_df, disease_col="main_name_list", alias_col="aliases_mapped", top_k=20)

In [8]:
#KG内に存在するentityかどうかのフラグ付与

KG_path = "/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/KG/cytoscape/merged_triples_all_v3.csv"
KG_df = (
    spark.read
         .csv(KG_path, header=True, inferSchema=True)
)

unique_entities_df = (
    KG_df.select(F.col("head_final").alias("entity"))
         .union(KG_df.select(F.col("tail_final").alias("entity")))
         .filter(F.col("entity").isNotNull() & (F.col("entity") != ""))
         .dropDuplicates()
)

unique_entities = [row.entity for row in unique_entities_df.orderBy("entity").collect()]

# オプション A：unique_entities を直接 isin で使う（サイズが許容できる場合）
# top_raw_with_flag = top_raw.withColumn(
#     "KG_flag",
#     F.col("alias").isin(unique_entities)
# )

# オプション B：大きい場合は Spark DF にしてジョイン
unique_entities_sdf = spark.createDataFrame([(e,) for e in unique_entities], ["entity"])

top_raw_with_flag = (
    top_raw.alias("t")
    .join(unique_entities_sdf.alias("k"), F.col("t.alias") == F.col("k.entity"), how="left")
    .withColumn("KG_flag", F.col("k.entity").isNotNull())
    .drop("entity")
)

top_raw_with_flag.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/top_aliases_by_main_name_raw.parquet")

In [9]:
from pyspark.sql import functions as F

top_raw = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/top_aliases_by_main_name_raw.parquet")

top_raw_grouped = (
    top_raw
    .withColumn("alias_tuple", F.struct(F.col("rank"), F.col("alias"), F.col("cnt"), F.col("KG_flag")))
    .groupBy("disease")
    .agg(
        F.sum("cnt").alias("alias_total_cnt"),
        F.array_sort(F.collect_list("alias_tuple")).alias("aliases_ranked")
    )
)
top_raw_grouped.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/top_aliases_by_main_name_raw_grouped.parquet")

## labelの調査

In [13]:
from pyspark.sql import functions as F
label = spark.read.parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/label_with_soap.parquet")

def disease_ranking(df, array_col, top_n=None):
    ranking = (
        df.select(F.explode_outer(F.col(array_col)).alias("disease"))
          .filter(F.col("disease").isNotNull())
          .groupBy("disease")
          .count()
          .orderBy(F.desc("count"), F.asc("disease"))
    )
    return ranking.limit(top_n) if top_n else ranking

rank_main    = disease_ranking(label, "main_name_list", top_n=500)
rank_suspect = disease_ranking(label, "suspect_name_list", top_n=500)
rank_acute   = disease_ranking(label, "acute_name_list", top_n=500)
rank_treat   = disease_ranking(label, "treat_name_list", top_n=500)

from functools import reduce

cols = ["main_name_list", "suspect_name_list", "acute_name_list", "treat_name_list"]

exploded_all = [
    label.select(F.explode_outer(F.col(c)).alias("disease"))
         .filter(F.col("disease").isNotNull())
    for c in cols
]

rank_total = reduce(lambda df1, df2: df1.unionByName(df2), exploded_all).groupBy("disease") \
    .count() \
    .orderBy(F.desc("count"), F.asc("disease"))


In [14]:
from pyspark.sql import functions as F

top_raw_grouped = (
    top_raw_grouped.alias("top")
    .join(rank_total.select("disease", F.col("count").alias("disease_count")), on="disease", how="left")
)

top_raw_grouped.show(truncate=False)


+------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|disease                 |alias_total_cnt|aliases_ranked                                                                                                                                                                                                                                                                            

In [22]:
from pyspark.sql import functions as F
disease_alias_list_df = top_raw_grouped.selectExpr(
    "disease",
    "disease_count",
    "transform(aliases_ranked, x -> x.alias) as aliases"
)


disease_alias_list_df.show(50, truncate=False)

+------------------------------------------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|disease                                               |disease_count|aliases                                                                                                                                                                                                                                                                                                          |
+------------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
disease_alias_list_df

In [15]:
top_raw_grouped.printSchema()

root
 |-- disease: string (nullable = true)
 |-- alias_total_cnt: long (nullable = true)
 |-- aliases_ranked: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- rank: integer (nullable = true)
 |    |    |-- alias: string (nullable = true)
 |    |    |-- cnt: long (nullable = true)
 |    |    |-- KG_flag: boolean (nullable = true)
 |-- disease_count: long (nullable = true)



In [16]:
top_raw_grouped.write.mode("overwrite").parquet("/Users/takami.soshi/Documents/GitHub/KGLLM_v2/analytics/project/shosin_disease_pred/test/top_aliases_by_main_name_raw_grouped.parquet")

In [17]:
top_raw_grouped

DataFrame[disease: string, alias_total_cnt: bigint, aliases_ranked: array<struct<rank:int,alias:string,cnt:bigint,KG_flag:boolean>>, disease_count: bigint]